In [4]:
import pandas as pd
import os
import shutil
from datetime import datetime

# === CONFIG ===
TRIPLES_FILE = "../data/kg/triples/movie_kg_triples.tsv"
OUTPUT_FILE = "../data/output/top10_recommended.txt"
BACKUP_DIR = "../data/output/backups"  # folder for backups
PREDICATE_REVIEW = "ex:predictedReview"
PREDICATE_NAME = "schema:name"
PREDICATE_YEAR = "schema:datePublished"

# ensure backup directory exists
os.makedirs(BACKUP_DIR, exist_ok=True)

# === LOAD ===
df = pd.read_csv(TRIPLES_FILE, sep="\t", header=None, names=["subject", "predicate", "object"])

# === STEP 1: filter predicted reviews ===
pred_df = df[df["predicate"] == PREDICATE_REVIEW].copy()
pred_df["score"] = pred_df["object"].str.extract(r"([\d.]+)").astype(float)

# === STEP 2: get movie titles ===
title_map = df[df["predicate"] == PREDICATE_NAME].set_index("subject")["object"].to_dict()

# === STEP 3: get release years ===
year_map = df[df["predicate"] == PREDICATE_YEAR].set_index("subject")["object"].to_dict()

# === STEP 4: enrich and sort ===
pred_df["title"] = pred_df["subject"].map(title_map)
pred_df["year"] = pred_df["subject"].map(year_map)

# clean year values like "published_1999" → "1999"
pred_df["year"] = pred_df["year"].str.extract(r"(\d{4})")

top10 = pred_df.sort_values("score", ascending=False).head(10)

# === FORMAT OUTPUT ===
lines = ["Recommended movies:"]
for _, row in top10.iterrows():
    if pd.notna(row["title"]) and pd.notna(row["year"]):
        lines.append(f"{row['title']} ({row['year']})")

# print to console
print("\n".join(lines))

# === HANDLE BACKUP ===
if os.path.exists(OUTPUT_FILE):
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    backup_file = os.path.join(BACKUP_DIR, f"top10_recommended_backup_{ts}.txt")
    shutil.move(OUTPUT_FILE, backup_file)
    print(f"Previous file backed up as {backup_file}")

# save new recommendations
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    f.write("\n".join(lines))

print(f"New recommendations saved to {OUTPUT_FILE}")

Recommended movies:
The Passion of the Christ (2004)
28 Weeks Later (2007)
Titanic (1997)
Black Panther (2018)
Wickie the Mighty Viking (2009)
Scary Stories to Tell in the Dark (2019)
Sweeney Todd: The Demon Barber of Fleet Street (2007)
Rudolph the Red-Nosed Reindeer: The Movie (1998)
The Watch (2012)
Jack and Jill (2011)
Previous file backed up as ../data/output/backups/top10_recommended_backup_20250921_151122.txt
New recommendations saved to ../data/output/top10_recommended.txt
